In [ ]:
import uuid
import os
from typing import List, Dict, Any
from dotenv import load_dotenv
import json
from bfcl.model_handler.oss_model.qwen import QwenHandler
from bfcl.model_handler.oss_model.llama import LlamaHandler
from llama_cpp import Llama
import requests
from utils import leggi_file_markdown,create_agent,Mediator,add_agents_to_mediator,send_and_print,validate_and_print

local_endpoint = "http://localhost:8080/v1/chat/completions"
qwen_1_5b_endpoint = "https://qczw73rk6noiiz27.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions" 
qwen_3b_endpoint = ""
qwen_7b_endpoint = ""
llama_3b_endpoint =""
llama_8b_endpoint ="https://l41j2vkrxy8txnck.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions"
huggingface_endpoint_url = local_endpoint

huggingface_endpoint_url = local_endpoint
MODEL_NAME_BFCL = 'LLama/Llama-3.2-3B-Instruct'
TEMPERATURE = 0.9

test_entry ={"id": "multiple_165", "question": [[{"role": "user", "content": "Find the price of a used Gibson Les Paul guitar in excellent condition in the Chicago area."}]], "function": [{"name": "identify_color_rgb", "description": "This function identifies the RGB values of a named color.", "parameters": {"type": "dict", "properties": {"color_name": {"type": "string", "description": "Name of the color."}, "standard": {"type": "string", "description": "The color standard (e.g. basic, pantone). Default is 'basic'"}}, "required": ["color_name"]}}, {"name": "board_game.chess.get_top_players", "description": "Find top chess players in a location based on rating.", "parameters": {"type": "dict", "properties": {"location": {"type": "string", "description": "The city you want to find the players from."}, "minimum_rating": {"type": "integer", "description": "Minimum rating to filter the players."}, "number_of_players": {"type": "integer", "default": 10, "description": "Number of players you want to retrieve, default value is 10"}}, "required": ["location", "minimum_rating"]}}, {"name": "guitar_price.find", "description": "Retrieve the price of a specific used guitar model based on its condition and location.", "parameters": {"type": "dict", "properties": {"model": {"type": "string", "description": "The model of the guitar."}, "condition": {"type": "string", "enum": ["Poor", "Good", "Excellent"], "description": "The condition of the guitar."}, "location": {"type": "string", "description": "The location where the guitar is being sold."}}, "required": ["model", "condition", "location"]}}]}


handler = LlamaHandler(model_name=MODEL_NAME_BFCL, temperature=0.9)

system_prompt_caller = leggi_file_markdown('system_prompt/caller.md',test_entry["function"])
messages = [
    {"role": "system", "content": system_prompt_caller},
    {"role": "user", "content": test_entry["question"][0][0]["content"]}
]

formatted_prompt = handler._format_prompt(messages, function=None)
print(formatted_prompt)

In [ ]:
from bfcl.v002_generator_bfcl_gpt import gpt_agent_generator
test_entry = {'question': 'Find me the best Italian restaurants in New York City with average customer ratings of more than 4 and accepts credit cards.', 'functions': [{'name': 'restaurant_search', 'description': 'Locates top rated restaurants based on specific criteria such as type of cuisine, ratings, and facilities. Note that the provided function is in Python 3 syntax.', 'parameters': {'type': 'dict', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. New York City, NY'}, 'cuisine': {'type': 'string', 'description': 'Preferred type of cuisine e.g., Italian, Indian, American, etc.'}, 'rating': {'type': 'integer', 'description': 'Minimum average customer rating out of 5'}, 'accepts_credit_cards': {'type': 'boolean', 'description': 'If the restaurant should accept credit cards.'}}, 'required': ['location', 'cuisine', 'rating', 'accepts_credit_cards']}}]}











MESSAGES OPENAI [{'role': 'system', 'content': 'You are tasked with generating a similar question based on the provided input. Additionally, you are provided a function definition to help understand the context or processing involved. Follow these rules:\n\n1. **Input Question Identification**:\n   - Identify the type of question (e.g., mathematical, logical, geometry, etc.).\n   - Preserve the same problem type in the output question (e.g., if the input is about calculating a factorial, the output must also be about factorials).\n\n2. **Generate Similar Questions**:\n   - Replace numeric values with different numbers while maintaining the same structure and context.\n   - Use the function definition as context to ensure the generated question aligns with the problem type but do not call the function.\n\n3. **Examples**:\n\n   **Example 1**:\n   Input Question: "Calculate the factorial of 2 using math functions."\n   Function: `math.factorial`\n   Output Question: "Calculate the factor